In [ ]:
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
import pandas as pd
import string
import gdown
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForSequenceClassification, AdamW

In [ ]:
gdown.download('https://drive.google.com/uc?export=download&id=18CHnzLHkElW707W4cvqeETe_kCveugN1', None, quiet=True)

'archive 39.zip'

In [ ]:
!unzip -qo "archive 39.zip" -d ./data

In [ ]:
train_data = pd.read_csv("/content/data/train.csv")
test_data = pd.read_csv("/content/data/test.csv")
device = torch.device("cuda")

train_data.head(), train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38740 entries, 0 to 38739
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   oid       38740 non-null  int64 
 1   category  38740 non-null  object
 2   text      38740 non-null  object
dtypes: int64(1), object(2)
memory usage: 908.1+ KB


(         oid      category                                               text
 0  365271984  winter_sport  Волшебные фото Виктория Поплавская ЕвгенияМедв...
 1  503385563       extreme  Возвращение в подземелье Треша 33 Эйфория тупо...
 2  146016084      football  Лучшие чешские вратари – Доминик Доминатор Гаш...
 3  933865449    boardgames  Rtokenoid Warhammer40k валрак решил нас подкор...
 4  713550145        hockey  Шестеркин затаскивает Рейнджерс в финал Восточ...,
 None)

In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

train_data['text'] = train_data['text'].apply(preprocess_text)

train_data = train_data.drop_duplicates(subset=['text'])

train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35745 entries, 0 to 38739
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   oid       35745 non-null  int64 
 1   category  35745 non-null  object
 2   text      35745 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_data['text'], train_data['category'], test_size=0.2, random_state=42
)

len(train_texts), len(val_texts)

(28596, 7149)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=512)

label_to_id = {label: idx for idx, label in enumerate(train_data['category'].unique())}
train_labels = [label_to_id[label] for label in train_labels]
val_labels = [label_to_id[label] for label in val_labels]

len(label_to_id), label_to_id

(13,
 {'winter_sport': 0,
  'extreme': 1,
  'football': 2,
  'boardgames': 3,
  'hockey': 4,
  'esport': 5,
  'athletics': 6,
  'motosport': 7,
  'basketball': 8,
  'tennis': 9,
  'autosport': 10,
  'martial_arts': 11,
  'volleyball': 12})

In [ ]:
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

In [ ]:
train_dataset = TextDataset(train_encodings, train_labels)
val_dataset = TextDataset(val_encodings, val_labels)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "DeepPavlov/rubert-base-cased",
    num_labels=len(label_to_id)
)

optimizer = AdamW(model.parameters(), lr=5e-5)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [ ]:
epochs = 4
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")

Epoch 1/4, Loss: 0.8422
Epoch 2/4, Loss: 0.3940
Epoch 3/4, Loss: 0.2378
Epoch 4/4, Loss: 0.1569


In [ ]:
model.save_pretrained("./fine_tuned_rubert")
tokenizer.save_pretrained("./fine_tuned_rubert")

('./fine_tuned_rubert/tokenizer_config.json',
 './fine_tuned_rubert/special_tokens_map.json',
 './fine_tuned_rubert/vocab.txt',
 './fine_tuned_rubert/added_tokens.json',
 './fine_tuned_rubert/tokenizer.json')

In [ ]:
def evaluate_model_combined(model, data_loader, device, id_to_label, max_examples=10):
    model.eval()
    correct = 0
    total = 0
    example_count = 0

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(outputs.logits, dim=1)

            correct += (predictions == labels).sum().item()
            total += labels.size(0)

            for text_id, pred in zip(batch['input_ids'], predictions):
                if example_count < max_examples:
                    text = tokenizer.decode(text_id, skip_special_tokens=True)
                    predicted_label = id_to_label[pred.item()]
                    print(f"Text: {text}\nPredicted Category: {predicted_label}\n")
                    example_count += 1

    accuracy = correct / total
    print(f"Accuracy: {accuracy:.4f}")
    return accuracy

In [ ]:
evaluate_model_combined(model, val_loader, device, {v: k for k, v in label_to_id.items()}, max_examples=10)

Text: кристиан хорнер макс ошибся у серхио была проблема с двигателем руководитель red btokenoid ratokenoid кристиан хорнер доволен результатами квалификации гран при австралии с учетом того что макс ферстаппен и серхио перес столкнулись с проблемами в финальном сегменте я очень доволен вторым и третьим местом на стартовой решетке результаты очень плотные и в разные моменты квалификации мне казалось что и макс и чеко могут выиграть поул – сказал хорнер в интервью sky sports хорнер объяснил что ферстаппену поула стоила ошибка – на своем самом быстром круге голландец заблокировал шины перес в свою очередь столкнулся с кратковременной потерей мощности силовой установки блокировка стоила максу около трех десятых секунды без нее он был бы очень очень близко ко времени шарля в первой попытке у чеко была небольшая потеря мощности к тому же у него в баке была лишняя десятая часть топлива потому что он хотел проехать три круга шарль проехал отличный круг но знаете ли мы все еще в первом и второ

0.8736886277801091